<a href="https://colab.research.google.com/github/harjeet88/google_collab/blob/main/spark_exmples/day5_leetcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!chmod -R +x /content/drive/MyDrive/installs/

In [ ]:
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']
# Verify installation
!java -version
!which java

openjdk version "11.0.2" 2019-01-15
OpenJDK Runtime Environment 18.9 (build 11.0.2+9)
OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
/content/drive/MyDrive/installs/jdk-11.0.2//bin/java


In [ ]:
import pyspark
import os
java_path = '/content/drive/MyDrive/installs/jdk-11.0.2/'
os.environ['JAVA_HOME'] = java_path
os.environ["SPARK_HOME"] = "drive/MyDrive/installs/spark-3.5.1-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/installs/py_package') # Replace with your actual path

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]

spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )

In [ ]:
spark

# Fix Names
https://leetcode.com/problems/fix-names-in-a-table/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/fix_names_1667.csv
user_id,name
1,aLice
2,bOB

Writing /content/drive/MyDrive/data/leetcode_sql180/fix_names_1667.csv


In [ ]:
from pyspark.sql.functions import *
from os.path import join
from ast import alias

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/fix_names_1667.csv",inferSchema=True,header=True)

In [ ]:
df.show()

+-------+-----+
|user_id| name|
+-------+-----+
|      1|aLice|
|      2|  bOB|
+-------+-----+



In [ ]:
df.withColumn("nameCap",initcap(col("name"))).show()

+-------+-----+-------+
|user_id| name|nameCap|
+-------+-----+-------+
|      1|aLice|  Alice|
|      2|  bOB|    Bob|
+-------+-----+-------+



In [ ]:
df1 = df.withColumn("nameCap",concat(substring(col("name"), 1, 1), \
                                     lower(substring(col("name"), 2,3))))

df1.show()

+-------+-----+-------+
|user_id| name|nameCap|
+-------+-----+-------+
|      1|aLice|   alic|
|      2|  bOB|    bob|
+-------+-----+-------+



In [ ]:
df.withColumn("length", length(col("name"))).show()

+-------+-----+------+
|user_id| name|length|
+-------+-----+------+
|      1|aLice|     5|
|      2|  bOB|     3|
+-------+-----+------+



#Patient conditions
https://leetcode.com/problems/patients-with-a-condition/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/patient_conditions_1527.csv
patient_id,patient_name,conditions
1,Daniel,YFEV COUGH
2,Alice,
3,Bob,DIAB100 MYOP
4,George,ACNE DIAB100
5,Alain,DIAB201

Writing /content/drive/MyDrive/data/leetcode_sql180/patient_conditions_1527.csv


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/patient_conditions_1527.csv",inferSchema=True,header=True)

In [ ]:
df1 = df.where(df.conditions.like("%DIAB1%"))
df1.show()

+----------+------------+------------+
|patient_id|patient_name|  conditions|
+----------+------------+------------+
|         3|         Bob|DIAB100 MYOP|
|         4|      George|ACNE DIAB100|
+----------+------------+------------+



In [ ]:
%%writefile /content/drive/MyDrive/data/leetcode_sql180/investment_2016_585.csv
pid,tiv_2015,tiv_2016,lat,lon
1,10,5,10,10
2,20,20,20,20
3,10,30,20,20
4,10,40,40,40

Writing /content/drive/MyDrive/data/leetcode_sql180/investment_2016_585.csv


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/data/leetcode_sql180/investment_2016_585.csv",inferSchema=True,header=True)

In [ ]:
df.show()

+---+--------+--------+---+---+
|pid|tiv_2015|tiv_2016|lat|lon|
+---+--------+--------+---+---+
|  1|      10|       5| 10| 10|
|  2|      20|      20| 20| 20|
|  3|      10|      30| 20| 20|
|  4|      10|      40| 40| 40|
+---+--------+--------+---+---+



In [ ]:
df.createOrReplaceTempView("investments")

In [ ]:
from pyspark.sql.functions import col,sum

In [ ]:
df1 = df.groupBy("lat","lon").count().filter(col("count") == 1).join(df,["lat","lon"],"inner")

In [ ]:
df2 = df1.groupBy("tiv_2015").count().filter(col("count") > 1).join(df1,["tiv_2015"],"inner")

In [ ]:
df2 = df2.withColumn("tiv_2016", col("tiv_2016").cast("int"))

In [ ]:
df2.select(sum(col("tiv_2016"))).show()

+-------------+
|sum(tiv_2016)|
+-------------+
|           45|
+-------------+

